## Comparing Accuracy: scikit-learn vs tensorflow

In this notebook, we train then test the model in a 60 / 40 split for the decision tree algo on both scikit-learn and tensorflow. First, we start with scikit-learn where we predict cloud vendor based on throughput.

In [1]:
import graphviz
import pandas
from sklearn import tree
from sklearn.model_selection import train_test_split

clf = tree.DecisionTreeClassifier()
input = pandas.read_csv("/home/glenn/git/clojure-news-feed/client/ml/etl/throughput.csv")
data = input[input.columns[6:9]]
target = input['cloud']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.4, random_state=0)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.937007874015748

Next, we evaluate scikit-learn accuracy where we predict feed implementation based on latency.

In [1]:
import graphviz
import pandas
from sklearn import tree
from sklearn.model_selection import train_test_split

clf = tree.DecisionTreeClassifier()
input = pandas.read_csv("/home/glenn/git/clojure-news-feed/client/ml/etl/latency.csv")
data = input[input.columns[7:9]]
data['cloud'] = input['cloud'].apply(lambda x: 1.0 if x == 'GKE' else 0.0)
target = input['feed']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.4, random_state=0)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/home/glenn/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0.9874015748031496

As you can see, scikit-learn has a 99% accuracy rate. We now do the same thing with tensorflow.

In [1]:
import tensorflow as tf
import numpy as np
import pandas
from tensorflow.python.ops import parsing_ops
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils
from sklearn.model_selection import train_test_split

input = pandas.read_csv("/home/glenn/git/clojure-news-feed/client/ml/etl/latency.csv")
data = input[input.columns[7:9]]
data['cloud'] = input['cloud'].apply(lambda x: 1.0 if x == 'GKE' else 0.0)
X_train, X_test, y_train, y_test = train_test_split(data, input['feed'], test_size=0.4, random_state=0)
X_train_np = np.array(X_train, dtype=np.float32)
y_train_np = np.array(y_train, dtype=np.int32)
X_test_np = np.array(X_test, dtype=np.float32)
y_test_np = np.array(y_test, dtype=np.int32)
hparams = tensor_forest.ForestHParams(num_classes=7,
                                      num_features=3,
                                      num_trees=1,
                                      regression=False,
                                      max_nodes=500).fill()
classifier = tf.contrib.tensor_forest.client.random_forest.TensorForestEstimator(hparams)
c = classifier.fit(x=X_train_np, y=y_train_np)
c.evaluate(x=X_test_np, y=y_test_np)


Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5a8068a4d0>, '_model_dir': '/tmp/tmpFG57Uw', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_environment'

/home/glenn/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Instructions for updating:
Please switch to the Estimator interface.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_output_columns': 8, 'feature_bagging_fraction': 1.0, 'valid_leaf_threshold': 1, 'checkpoint_stats': False, 'initialize_average_splits': False, 'pruning_type': 0, 'prune_every_samples': 0, 'dominate_fraction': 0.99, 'max_fertile_nodes': 0, 'early_finish_check_every_samples': 0, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'param_file': None, 'bagged_num_features': 3, 'use_running_stats_method': False, 'max_nodes': 500, 'split_finish_name': 'basic', 'leaf_model_type': 0, 'stats_model_typ

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-08-07:13:54
INFO:tensorflow:Saving dict for global step 317: accuracy = 0.97952753, global_step = 317, loss = 1.1858935
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


{'accuracy': 0.97952753, 'global_step': 317, 'loss': 1.1858935}

Looks like tensorflow has a 98% accuracy rate which is 1% less than scikit-learn algo.